In [36]:
# unzip input file first

import xml.etree.ElementTree as ET
import requests
import math

In [2]:
root = ET.parse('input-files/RoughlyTravisCounty.osm').getroot()

In [8]:
latitudes = []
longitudes = []
for type_tag in root.findall("node"):
    d = type_tag.attrib
    latitudes.append(float(d["lat"]))
    longitudes.append(float(d["lon"]))

In [50]:
LATITUDE_MILES = 69
LONGITUDE_MILES = 54.6

vertical_distance = (max(latitudes) - min(latitudes))*LATITUDE_MILES
horizontal_distance = (max(longitudes) - min(longitudes))*LONGITUDE_MILES

In [51]:
# Assuming we want to make 400 squares, how should we partition the vertically/horizontally
NUM_REGIONS = 400
square_len = (vertical_distance*horizontal_distance/NUM_REGIONS)**0.5

In [133]:
latitude_delta = square_len/LATITUDE_MILES
longitude_delta = square_len/LONGITUDE_MILES

In [53]:
lat_lon = zip(latitudes, longitudes)

In [62]:
min_lat = min(latitudes)
min_lon = min(longitudes)
max_lat = max(latitudes)
max_lon = max(longitudes)
nrows = math.ceil((max_lat - min_lat) / latitude_delta)
ncols = math.ceil((max_lon - min_lon) / longitude_delta)
def map_point_to_region(latitude, longitude):
    return math.floor((latitude-min_lat)/latitude_delta) * ncols  + math.floor((longitude-min_lon)/longitude_delta)

In [88]:
def get_representative(region_num): # 8 cols per row, # R10 --> row_num = 1, col_num = 2
    row_num = region_num//ncols # Correct
    col_num = region_num - row_num*ncols
    lat = min_lat + row_num * latitude_delta + 0.5*latitude_delta
    lon = min_lon + col_num * longitude_delta + 0.5*longitude_delta
    return [lon, lat]

In [89]:
map_point_to_region(30.084446, -97.702885)

13

In [90]:
get_representative(13)

[-97.68468325435288, 30.090783782593846]

In [67]:
for i in range(400):
    get_representative(i)

0.030993565187689837

In [91]:
 headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf624884518d9d4075408e95e3401165976247',
    'Content-Type': 'application/json; charset=utf-8'
}

In [109]:
result = [[] for i in range(400)]
for num in range(50):
    body = {"locations": [get_representative(i) for i in range(400)], "destinations": [i for i in range(8*num, 8*num+8)]}
    call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)
    for i in range(400):
        result[i] += call.json()["durations"][i]
    print("Done with iteration {}".format(num))

Done with iteration 0
Done with iteration 1
Done with iteration 2
Done with iteration 3
Done with iteration 4
Done with iteration 5
Done with iteration 6
Done with iteration 7
Done with iteration 8
Done with iteration 9
Done with iteration 10
Done with iteration 11
Done with iteration 12
Done with iteration 13
Done with iteration 14
Done with iteration 15
Done with iteration 16
Done with iteration 17
Done with iteration 18
Done with iteration 19
Done with iteration 20
Done with iteration 21
Done with iteration 22
Done with iteration 23
Done with iteration 24
Done with iteration 25
Done with iteration 26
Done with iteration 27
Done with iteration 28
Done with iteration 29
Done with iteration 30
Done with iteration 31
Done with iteration 32
Done with iteration 33
Done with iteration 34
Done with iteration 35
Done with iteration 36
Done with iteration 37
Done with iteration 38
Done with iteration 39
Done with iteration 40
Done with iteration 41
Done with iteration 42
Done with iteration 4

In [128]:
import csv
with open("save_distances.csv", "w") as c:
    w = csv.writer(c)
    for r in result:
        w.writerow(r)

In [134]:
import json

d = {"latitude_min": min_lat, "longitude_min": min_lon, "latitude_step": latitude_delta, "longitude_step": longitude_delta}
d[""]
with open("grid_info.json", "w") as j:
    json.dump(d, j)

SyntaxError: invalid syntax (<ipython-input-134-a39273ed45c2>, line 4)